<a href="https://colab.research.google.com/github/nitinware/GoogleCloabNotebooks/blob/master/run_mlflow_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow

     |████████████████████████████████| 14.4 MB 57 kB/s 
     |████████████████████████████████| 146 kB 59.6 MB/s 
     |████████████████████████████████| 636 kB 41.0 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 79 kB 6.0 MB/s 
     |████████████████████████████████| 1.1 MB 39.0 MB/s 
     |████████████████████████████████| 170 kB 51.7 MB/s 
     |████████████████████████████████| 75 kB 3.5 MB/s 
     |████████████████████████████████| 68 kB 5.3 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158170 sha256=2d32432a9a69c01718b2702ac7d80d94ab71f60a8285302de3aa6814d4914090
  Stored in directory: /root/.cache/pip/wheels/be/5d/0a/9e13f53f4f5dfb67cd8d245bb7cdffe12f135846f491a283e3
  Created wheel for databricks-cli: filename=databricks_cli-0.14.3-py3-none-any.whl size=100557 sha256=6d153a447bee211b3570449c510d6d090d80eba8898543acff4f35598f5ec163

In [2]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-py3-none-any.whl size=19262 sha256=c5fefefcd04d2516cbac424e1c5448f850391984d657a46b24feb90c6a55c98e
  Stored in directory: /root/.cache/pip/wheels/1b/f7/72/35c95a53d15b91dd00df6cf1304d49a31ec5ed6f954c2d4e32
Successfully built pyngrok


In [4]:
get_ipython().system_raw("mlflow ui --port 5000 &")

In [5]:
from pyngrok import ngrok

In [6]:
ngrok.kill()

In [7]:
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)

In [8]:
print("MLFlow UI ", ngrok_tunnel.public_url)

MLFlow UI  https://4ed7b55cc923.ngrok.io


In [9]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn

In [10]:
!ls

mlruns	sample_data  storepurchasedata.csv


In [11]:
mlflow.set_experiment(experiment_name="mlflow demo")

INFO: 'mlflow demo' does not exist. Creating a new experiment


In [12]:
training_data = pd.read_csv('storepurchasedata.csv')
print("Loaded training data")

Loaded training data


In [13]:
 mlflow.log_param("training percentage", 70)
mlflow.log_param("dataset shape", training_data.shape)
    
X = training_data.iloc[:, :-1].values
y = training_data.iloc[:, -1].values

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.70, random_state=0)

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [18]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski')    
classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [19]:
y_pred = classifier.predict(X_test)    
y_prob = classifier.predict_proba(X_test)[:, 1]

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [21]:
from sklearn.metrics import accuracy_score    
model_accuracy = accuracy_score(y_test, y_pred)

In [22]:
mlflow.log_metric("accuracy", model_accuracy)

In [23]:
mlflow.sklearn.log_model(classifier, "model")